In [0]:
import pandas as pd
import seaborn as sns
#
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
#
import mlflow
from mlflow.models.signature import infer_signature
#
import logging
import json 
import os
from sys import version_info

In [0]:
logging.getLogger("mlflow").setLevel(logging.FATAL)

In [0]:
diamonds_df = sns.load_dataset('diamonds').drop(['cut', 'color', 'clarity'], axis=1)
#
X_train, X_test, y_train, y_test = train_test_split(diamonds_df.drop(["price"], axis=1), diamonds_df["price"], random_state=42)

X_train.head()

### Sklearn model

In [0]:
class sklearn_model(mlflow.pyfunc.PythonModel):
    def __init__(self, params):
        self.params = params
        self.rf_model = None
        self.config = None


    def load_context(self,context = None,config_path=None):
        """ When loading a pyfunc, this method runs automatically with the related
        context. This method is designed to perform the same functionality when
        run in a notebook or a downstream operation (like a REST endpoint).
        If the `context` object is provided, it will load the path to a config from 
        that object (this happens with `mlflow.pyfunc.load_model()` is called).
        If the `config_path` argument is provided instead, it uses this argument
        in order to load in the config. """
        #
        if context: # This block executes for server run
            config_path = context.artifacts["config_path"]
        else:       # This block executes for notebook run
            pass
        self.config = json.load(open(config_path))

    def fit(self, X_train, y_train):
        rf_model = RandomForestRegressor(**self.params)
        rf_model.fit(X_train, y_train)
        self.rf_model = rf_model

    def predict(self, context, model_input):
        return self.rf_model.predict(model_input)


In [0]:
params_sklearn = {
    "n_estimators": 15, 
    "max_depth": 5
}
#
# Designate a path
config_path_sklearn = "data_sklearn.json"
#
# Save the results
with open(config_path_sklearn, "w") as f:
    json.dump(params_sklearn, f)
#
# Generate an artifact object to saved
# All paths to the associated values will be copied over when saving
artifacts_sklearn = {"config_path": config_path_sklearn}

In [0]:
model_sk = sklearn_model(params_sklearn)
#
model_sk.load_context(config_path=config_path_sklearn) 
#
# Confirm the config has loaded
model_sk.config

In [0]:
model_sk.fit(X_train, y_train)

In [0]:
predictions_sklearn = model_sk.predict(context=None, model_input=X_test)
pd.DataFrame({'actual prices': list(y_test), 'predictions': list(predictions_sklearn)}).head(5)

In [0]:
signature_sklearn = infer_signature(X_test, predictions_sklearn)
signature_sklearn

In [0]:
conda_env_sklearn = {
    "channels": ["defaults"],
    "dependencies": [
        f"python={version_info.major}.{version_info.minor}.{version_info.micro}",
        "pip",
        {"pip": ["mlflow",
                 f"scikit-learn=={sklearn.__version__}"]
        },
    ],
    "name": "sklearn_env"
}
conda_env_sklearn

In [0]:
with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        "sklearn_RFR", 
        python_model=model_sk, 
        artifacts=artifacts_sklearn,
        conda_env=conda_env_sklearn,
        signature=signature_sklearn,
        input_example=X_test[:3] 
  )

In [0]:
mlflow_pyfunc_model_path_sk = f"runs:/{run.info.run_id}/sklearn_RFR"
print("model_uri : ",mlflow_pyfunc_model_path_sk)
loaded_preprocess_model_sk = mlflow.pyfunc.load_model(mlflow_pyfunc_model_path_sk)

#### Tensorflow model

In [0]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Normalization

print("TF version : ",tf.__version__)

    

In [0]:
print("X_train : ",X_train.shape)
print("y_train : ",y_train.shape)

In [0]:
class custom_tf_model(mlflow.pyfunc.PythonModel):
    def __init__(self, params):
        self.params = params
        self.tf_model = None
        self.config = None


    def load_context(self,context = None,config_path=None):
        """ When loading a pyfunc, this method runs automatically with the related
        context. This method is designed to perform the same functionality when
        run in a notebook or a downstream operation (like a REST endpoint).
        If the `context` object is provided, it will load the path to a config from 
        that object (this happens with `mlflow.pyfunc.load_model()` is called).
        If the `config_path` argument is provided instead, it uses this argument
        in order to load in the config. """
        #
        if context: # This block executes for server run
            config_path = context.artifacts["config_path"]
            #self.tf_model = tf.keras.models.load_model(config_path)
        else:       # This block executes for notebook run
            pass
        self.config = json.load(open(config_path))

    def fit(self, X_train, y_train):
        tf_model = self.buildAndCompile_tf_regression_mode(input_shape=self.params["input_shape"])
        tf_model.fit(X_train,y_train,epochs= self.params["epoch"])
        self.tf_model = tf_model

    def predict(self, context, model_input):
        return self.tf_model.predict(model_input)
    

    def buildAndCompile_tf_regression_mode(self, input_shape):
        model = Sequential([
            Normalization(input_shape=input_shape),
            Dense(64, activation='relu'),
            Dense(64, activation='relu'),
            Dense(1)
        ])

        model.compile(loss='mean_absolute_error',
                        optimizer=tf.keras.optimizers.Adam(0.001))
        return model
    

    


In [0]:
params_tf = {
    "input_shape": [6], 
    "epoch": 3
}

config_path_tf = "data_tf.json"

# Save the results
with open(config_path_tf, "w") as f:
    json.dump(params_tf, f)
artifacts_tf = {"config_path": config_path_tf} 

In [0]:
model_tf = custom_tf_model(params_tf)
#
model_tf.load_context(config_path=config_path_tf) 
#
# Confirm the config has loaded
model_tf.config

In [0]:
model_tf.fit(X_train,y_train)

In [0]:
conda_env_tf = {
    "channels": ["defaults"],
    "dependencies": [
        f"python={version_info.major}.{version_info.minor}.{version_info.micro}",
        "pip",
        {"pip": ["mlflow",
                 f"tensorflow=={tf.__version__}"]
        },
    ],
    "name": "tf_env"
}
conda_env_tf

In [0]:
predictions_tf = model_tf.predict(context=None, model_input=X_test)
pd.DataFrame({'actual prices': list(y_test), 'predictions': list(predictions_tf)}).head(5)


In [0]:
signature_tf = infer_signature(X_test, predictions_tf)
signature_tf

In [0]:
artifacts_tf

In [0]:
with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        "tf_regressor", 
        python_model=model_tf, 
        artifacts=artifacts_tf,
        conda_env=conda_env_tf,
        signature=signature_tf,
        input_example=X_test[:3] 
  )

In [0]:
mlflow_pyfunc_model_path_tf = f"runs:/{run.info.run_id}/tf_regressor"
print("mlflow_pyfunc_model_path_tf : ",mlflow_pyfunc_model_path_tf)
loaded_preprocess_model_tf = mlflow.pyfunc.load_model(mlflow_pyfunc_model_path_tf)


In [0]:
#
y_pred_tf = loaded_preprocess_model_tf.predict(X_test)
#
pd.DataFrame({'actual prices': list(y_test), 'predictions': list(y_pred_tf)}).head(5)